In [ ]:
import numpy as np
import torch as T
import torch.nn.functional as F
from tqdm import trange

In [ ]:
def eucl(x, y):
    return T.linalg.norm(F.normalize(x) - F.normalize(y), dim=-1).item()

def cos(x, y):
    return F.cosine_similarity(x, y).item()

### Claim 1: Cosine Similarity and $L_2$-normalized Euclidean Distance preserve relative ordering

Let $x, y, z \in \mathbb{R}^d$, with unit norm, then: $\cos(x, y) \geq \cos(x, z) \iff eucl(x, y) < eucl(x, z)$

**Proof:**

First, note that $eucl(x, y)^2 = \sum_{i=1}^d (x_i - y_i)^2 = \sum_{i=1}^d x_i^2 + y_i^2 - 2 \cdot x_i y_i = ||x||_2^2 + ||y||_2^2 - 2 \cdot x^\top y = 2 - 2 \cdot \cos(x, y)$

$(\Rightarrow): \cos(x, y) \geq \cos(x, z) \implies 2 -2 \cdot \cos(x, y) < 2 -2 \cdot \cos(x, z) \implies eucl(x,y)^2 < eucl(x, z)^2 \implies eucl(x, y) < eucl(x, z)$

$(\Leftarrow): eucl(x, y) < eucl(x, z) \implies 2 - 2\cdot \cos(x, y) < 2 -2 \cdot \cos(x, z) \implies \cos(x, y) \geq \cos(x, z)$

$\blacksquare$

In [ ]:
N = 100_000

for _ in trange(N):
    d = np.random.randint(2, 128 + 1)
    p = np.round(np.random.uniform(), 2)
    
    x, y, z = (T.rand(3, 1, d) - 0.5) * T.bernoulli(p * T.ones(3, 1, d))
    
    if not T.any(x) or not T.any(y) or not T.any(z):
        continue
    
    cxy = cos(x, y)
    cxz = cos(x, z)
    exy = eucl(x, y)
    exz = eucl(x, z)
    
    lhs = True if np.isclose(cxy, cxz, rtol=0, atol=1e-6) else cxy > cxz
    rhs = True if np.isclose(exy, exz, rtol=0, atol=1e-6) else exy < exz
    
    if lhs != rhs:
        print(f"{cxy=}, {cxz=}")
        print(f"{exy=}, {exz=}")
        raise ValueError()

### Claim 2: (1 - Cosine Similarity) is not a proper distance metric

The triangle inequality does not always hold, meaning that:

$(\exists) \ x, y, z \in \mathbb{R}^d$ such that $1-\cos(x,y) > 1-\cos(x,z)+1-\cos(z,y)$, or

$1 + \cos(x, y) \leq \cos(x, z) + \cos(z, y)$

Simple counterexample:
$x = \langle 1, 0 \rangle, y = \langle 0, 1 \rangle, z = \langle (0, 1), (0, 1) \rangle$

This means that $x \bot y$ and $z$ is between them, such that $\angle(x,z) = \theta \implies \angle(z, y) = \frac{\pi}{2} - \theta$

$1 + \cos(x,y) \leq \cos(x, z) + \cos(z, y)$

$1 + 0 \leq \cos(\theta) + \cos(\frac{\pi}{2} - \theta)$

$1 \leq \cos(\theta) + \sin(\theta)$

$1 \leq \cos^2(\theta) + \sin^2(\theta) + \sin(2\theta)$

$\sin(2\theta) \geq 0 \implies \theta \in [0, \frac{\pi}{2}]$

---

Let $\theta = \frac{\pi}{4}$, then

1. $\cos(x,y) \leq \cos(x, z) + \cos(z, y) \implies 0 \leq \frac{\sqrt{2}}{2} + \frac{\sqrt{2}}{2}$ (True)

2. $1-\cos(x,y) \leq 1-\cos(x,z) +1-\cos(z,y) \implies 1 \leq 2 \cdot (1 - \frac{\sqrt{2}}{2}) = 2 - \sqrt{2} \approx 0.58578$ (False)

### Follow-up: Where is triangle inequality needed?